In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'cryptoArbitrage.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
import ccxt,pandas as pd
import numpy as np
from core.models import (
    Exchange,
    CryptoCurrencyPair
)

In [9]:
exchange_list = ["Binance", "KuCoin", "MEXC", "ByBit", "digifinex", "bitforex", "Kraken", "Phemex", "YoBit"]
exchange_list = list(map(lambda x : x.lower(),exchange_list))
exchange_dict = {}
for i in exchange_list:
    try:
        exchange_name = getattr(ccxt,i)()
        exchange_name.load_markets()
        exchange_dict[i] = exchange_name.symbols
    except Exception as exception:
        pass

In [12]:
for i in exchange_list:
    Exchange.objects.create(name = i)

In [20]:
my_list = [i for i in exchange_dict.values()]

In [29]:
for k,v in exchange_dict.items():
    print(len(v))

2646
1244
2265
405
498
664
750


In [35]:
from django.db.models import OuterRef
from django.db.models import (
    Count,
    F,
    Sum,
    OuterRef,
    Value,
    IntegerField,
    Prefetch,
    Q,
    ExpressionWrapper,
    functions,
    CharField,
    Case,
    When
)


In [40]:
Exchange.objects.annotate(
    count = CryptoCurrencyPair.objects.filter(
        exchange_id = OuterRef("uid")
    ).values(
        "exchange_id"
    ).annotate(
        count = Count("exchange_id")
    ).values("count")).values("count")

<QuerySet [{'count': 2646}, {'count': 1244}, {'count': 2265}, {'count': None}, {'count': 405}, {'count': 498}, {'count': 664}, {'count': 750}, {'count': None}]>

In [60]:
CryptoCurrencyPair.objects.only("symbol").values_list("symbol").distinct("symbol")

<QuerySet [('1000000BABYDOGE/USDT',), ('1000000 NFT/USD:USD',), ('1000000PIT/USDT',), ('100000AIDOGE/USDT:USDT',), ('100000BABYDOGE/USDT:USDT',), ('100000 SHIB/USD:USD',), ('10000LADYS/USDT',), ('10000 LADYS/USDT:USDT',), ('1000BEN/USDT:USDT',), ('1000BONK/USDT',), ('1000BONK/USDT:USDT',), ('1000BRISE/USDT',), ('1000BTT/USDT:USDT',), ('1000CAW/USDT:USDT',), ('1000 DENT/USD:USD',), ('1000DOGE2/USDT:USDT',), ('1000FLOKI/USDT',), ('1000 FLOKI/USDT:USDT',), ('1000FLOKI/USDT:USDT',), ('1000 HOT/USD:USD',), '...(remaining elements truncated)...']>

In [15]:
list(CryptoCurrencyPair.objects.filter(symbol__exact = "BTC/USDT").values_list("exchange__name",flat=True))

['binance', 'kucoin', 'mexc', 'digifinex', 'bitforex', 'kraken', 'phemex']